In [93]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [94]:
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [95]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(labels = ['EIN', 'NAME'], axis = 1, inplace = True)

In [96]:
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [97]:
# Determine which values to replace if counts are less than ...?
replace_application = application_df.APPLICATION_TYPE.value_counts()\
.loc[application_df.APPLICATION_TYPE.value_counts() < 200].index

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [98]:
application_df.CLASSIFICATION.value_counts().head(20)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64

In [99]:
# Determine which values to replace if counts are less than ..?
replace_class = application_df.CLASSIFICATION.value_counts()\
.loc[application_df.CLASSIFICATION.value_counts() < 500].index

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [100]:
# Generate our categorical variable lists
# STATUS and IS_SUCCESSFUL are also categorical but are already encoded
application_cat=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE',
                 'ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS']

In [101]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(X=application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/Users/jsaizlomas/miniforge3/envs/propensity2sell/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [102]:
encode_df.shape

(34299, 42)

In [103]:
#§ Merge one-hot encoded features and drop the originals
application_df = pd.concat([application_df.drop(labels=application_cat,axis=1), encode_df], axis=1)

In [104]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(labels=['IS_SUCCESSFUL'],axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=17)

In [105]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train.drop(labels='ASK_AMT', axis=1))

# Scale the data
X_train_scaled = X_scaler.transform(X_train.drop(labels='ASK_AMT', axis=1))
X_test_scaled = X_scaler.transform(X_test.drop(labels='ASK_AMT', axis=1))

In [106]:
X_train_scaled.shape

(22980, 43)

In [137]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(60, activation='relu'))
nn.add(tf.keras.layers.Dense(60, activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(1, activation='sigmoid'))
nn.build(X_train_scaled.shape)

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (22980, 60)               2640      
                                                                 
 dense_42 (Dense)            (22980, 60)               3660      
                                                                 
 dense_43 (Dense)            (22980, 1)                61        
                                                                 
Total params: 6,361
Trainable params: 6,361
Non-trainable params: 0
_________________________________________________________________


In [138]:
# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [139]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="saved_models_opt/weights.{epoch:02d}.hdf5",
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [140]:
# Train the model
nn.fit(x=X_train_scaled, batch_size=16, y=y_train,validation_data=(X_test_scaled, y_test), epochs=500,)

Epoch 1/500
1437/1437 [==============================] - 2s 983us/step - loss: 0.5692 - accuracy: 0.7209 - val_loss: 0.5630 - val_accuracy: 0.7253
Epoch 2/500
1437/1437 [==============================] - 1s 942us/step - loss: 0.5548 - accuracy: 0.7296 - val_loss: 0.5609 - val_accuracy: 0.7255
Epoch 3/500
1437/1437 [==============================] - 1s 936us/step - loss: 0.5527 - accuracy: 0.7296 - val_loss: 0.5571 - val_accuracy: 0.7267
Epoch 4/500
1437/1437 [==============================] - 1s 948us/step - loss: 0.5508 - accuracy: 0.7285 - val_loss: 0.5571 - val_accuracy: 0.7277
Epoch 5/500
1437/1437 [==============================] - 1s 938us/step - loss: 0.5488 - accuracy: 0.7298 - val_loss: 0.5595 - val_accuracy: 0.7274
Epoch 6/500
1437/1437 [==============================] - 1s 944us/step - loss: 0.5479 - accuracy: 0.7315 - val_loss: 0.5575 - val_accuracy: 0.7272
Epoch 7/500
1437/1437 [==============================] - 1s 984us/step - loss: 0.5471 - accuracy: 0.7309 - val_loss: 0

1437/1437 [==============================] - 1s 922us/step - loss: 0.5308 - accuracy: 0.7397 - val_loss: 0.5881 - val_accuracy: 0.7304
Epoch 112/500
1437/1437 [==============================] - 1s 909us/step - loss: 0.5314 - accuracy: 0.7398 - val_loss: 0.5849 - val_accuracy: 0.7299
Epoch 113/500
1437/1437 [==============================] - 1s 908us/step - loss: 0.5311 - accuracy: 0.7408 - val_loss: 0.5927 - val_accuracy: 0.7298
Epoch 114/500
1437/1437 [==============================] - 1s 918us/step - loss: 0.5318 - accuracy: 0.7399 - val_loss: 0.5924 - val_accuracy: 0.7297
Epoch 115/500
1437/1437 [==============================] - 1s 917us/step - loss: 0.5312 - accuracy: 0.7401 - val_loss: 0.5940 - val_accuracy: 0.7301
Epoch 116/500
1437/1437 [==============================] - 1s 919us/step - loss: 0.5313 - accuracy: 0.7405 - val_loss: 0.5901 - val_accuracy: 0.7308
Epoch 117/500
1437/1437 [==============================] - 1s 918us/step - loss: 0.5306 - accuracy: 0.7404 - val_loss: 0

1437/1437 [==============================] - 1s 922us/step - loss: 0.5312 - accuracy: 0.7406 - val_loss: 0.6274 - val_accuracy: 0.7296
Epoch 167/500
1437/1437 [==============================] - 1s 914us/step - loss: 0.5304 - accuracy: 0.7406 - val_loss: 0.6116 - val_accuracy: 0.7290
Epoch 168/500
1437/1437 [==============================] - 1s 915us/step - loss: 0.5313 - accuracy: 0.7404 - val_loss: 0.6132 - val_accuracy: 0.7297
Epoch 169/500
1437/1437 [==============================] - 1s 916us/step - loss: 0.5299 - accuracy: 0.7407 - val_loss: 0.6174 - val_accuracy: 0.7287
Epoch 170/500
1437/1437 [==============================] - 1s 926us/step - loss: 0.5311 - accuracy: 0.7414 - val_loss: 0.6036 - val_accuracy: 0.7283
Epoch 171/500
1437/1437 [==============================] - 1s 918us/step - loss: 0.5298 - accuracy: 0.7409 - val_loss: 0.6064 - val_accuracy: 0.7285
Epoch 172/500
1437/1437 [==============================] - 1s 907us/step - loss: 0.5304 - accuracy: 0.7408 - val_loss: 0

1437/1437 [==============================] - 1s 922us/step - loss: 0.5295 - accuracy: 0.7414 - val_loss: 0.6322 - val_accuracy: 0.7291
Epoch 222/500
1437/1437 [==============================] - 1s 918us/step - loss: 0.5295 - accuracy: 0.7409 - val_loss: 0.6191 - val_accuracy: 0.7299
Epoch 223/500
1437/1437 [==============================] - 1s 923us/step - loss: 0.5292 - accuracy: 0.7413 - val_loss: 0.6175 - val_accuracy: 0.7274
Epoch 224/500
1437/1437 [==============================] - 1s 919us/step - loss: 0.5289 - accuracy: 0.7412 - val_loss: 0.6341 - val_accuracy: 0.7287
Epoch 225/500
1437/1437 [==============================] - 1s 918us/step - loss: 0.5303 - accuracy: 0.7400 - val_loss: 0.6352 - val_accuracy: 0.7274
Epoch 226/500
1437/1437 [==============================] - 1s 928us/step - loss: 0.5405 - accuracy: 0.7409 - val_loss: 0.6150 - val_accuracy: 0.7275
Epoch 227/500
1437/1437 [==============================] - 1s 919us/step - loss: 0.5289 - accuracy: 0.7412 - val_loss: 0

1437/1437 [==============================] - 1s 892us/step - loss: 0.5283 - accuracy: 0.7409 - val_loss: 0.6111 - val_accuracy: 0.7290
Epoch 277/500
1437/1437 [==============================] - 1s 887us/step - loss: 0.5290 - accuracy: 0.7417 - val_loss: 0.6142 - val_accuracy: 0.7289
Epoch 278/500
1437/1437 [==============================] - 1s 887us/step - loss: 0.5293 - accuracy: 0.7412 - val_loss: 0.6292 - val_accuracy: 0.7290
Epoch 279/500
1437/1437 [==============================] - 1s 886us/step - loss: 0.5353 - accuracy: 0.7411 - val_loss: 0.6704 - val_accuracy: 0.7284
Epoch 280/500
1437/1437 [==============================] - 1s 888us/step - loss: 0.5285 - accuracy: 0.7415 - val_loss: 0.6776 - val_accuracy: 0.7293
Epoch 281/500
1437/1437 [==============================] - 1s 936us/step - loss: 0.5286 - accuracy: 0.7418 - val_loss: 0.6578 - val_accuracy: 0.7293
Epoch 282/500
1437/1437 [==============================] - 1s 890us/step - loss: 0.5289 - accuracy: 0.7413 - val_loss: 0

1437/1437 [==============================] - 1s 919us/step - loss: 0.5295 - accuracy: 0.7413 - val_loss: 0.6374 - val_accuracy: 0.7287
Epoch 332/500
1437/1437 [==============================] - 1s 920us/step - loss: 0.5286 - accuracy: 0.7418 - val_loss: 0.6485 - val_accuracy: 0.7290
Epoch 333/500
1437/1437 [==============================] - 1s 910us/step - loss: 0.5285 - accuracy: 0.7419 - val_loss: 0.6272 - val_accuracy: 0.7298
Epoch 334/500
1437/1437 [==============================] - 1s 910us/step - loss: 0.5286 - accuracy: 0.7412 - val_loss: 0.6535 - val_accuracy: 0.7287
Epoch 335/500
1437/1437 [==============================] - 1s 912us/step - loss: 0.5292 - accuracy: 0.7413 - val_loss: 0.6817 - val_accuracy: 0.7288
Epoch 336/500
1437/1437 [==============================] - 1s 912us/step - loss: 0.5295 - accuracy: 0.7410 - val_loss: 0.6432 - val_accuracy: 0.7294
Epoch 337/500
1437/1437 [==============================] - 1s 971us/step - loss: 0.5287 - accuracy: 0.7413 - val_loss: 0

1437/1437 [==============================] - 1s 896us/step - loss: 0.5286 - accuracy: 0.7408 - val_loss: 0.6226 - val_accuracy: 0.7287
Epoch 387/500
1437/1437 [==============================] - 1s 896us/step - loss: 0.5285 - accuracy: 0.7419 - val_loss: 0.6254 - val_accuracy: 0.7295
Epoch 388/500
1437/1437 [==============================] - 1s 896us/step - loss: 0.5287 - accuracy: 0.7415 - val_loss: 0.6426 - val_accuracy: 0.7291
Epoch 389/500
1437/1437 [==============================] - 1s 897us/step - loss: 0.5283 - accuracy: 0.7411 - val_loss: 0.6636 - val_accuracy: 0.7290
Epoch 390/500
1437/1437 [==============================] - 1s 900us/step - loss: 0.5288 - accuracy: 0.7411 - val_loss: 0.6487 - val_accuracy: 0.7289
Epoch 391/500
1437/1437 [==============================] - 1s 903us/step - loss: 0.5296 - accuracy: 0.7415 - val_loss: 0.6356 - val_accuracy: 0.7291
Epoch 392/500
1437/1437 [==============================] - 1s 896us/step - loss: 0.5286 - accuracy: 0.7413 - val_loss: 0

1437/1437 [==============================] - 1s 907us/step - loss: 0.5488 - accuracy: 0.7416 - val_loss: 0.9144 - val_accuracy: 0.7291
Epoch 442/500
1437/1437 [==============================] - 1s 911us/step - loss: 0.5366 - accuracy: 0.7413 - val_loss: 0.6729 - val_accuracy: 0.7282
Epoch 443/500
1437/1437 [==============================] - 1s 911us/step - loss: 0.5279 - accuracy: 0.7415 - val_loss: 0.6715 - val_accuracy: 0.7288
Epoch 444/500
1437/1437 [==============================] - 1s 900us/step - loss: 0.5279 - accuracy: 0.7420 - val_loss: 0.6625 - val_accuracy: 0.7291
Epoch 445/500
1437/1437 [==============================] - 1s 902us/step - loss: 0.5283 - accuracy: 0.7418 - val_loss: 0.6666 - val_accuracy: 0.7288
Epoch 446/500
1437/1437 [==============================] - 1s 914us/step - loss: 0.5280 - accuracy: 0.7415 - val_loss: 0.6735 - val_accuracy: 0.7282
Epoch 447/500
1437/1437 [==============================] - 1s 918us/step - loss: 0.5281 - accuracy: 0.7413 - val_loss: 0

1437/1437 [==============================] - 1s 912us/step - loss: 0.5404 - accuracy: 0.7409 - val_loss: 0.9321 - val_accuracy: 0.7284
Epoch 497/500
1437/1437 [==============================] - 1s 909us/step - loss: 0.5359 - accuracy: 0.7416 - val_loss: 0.6282 - val_accuracy: 0.7292
Epoch 498/500
1437/1437 [==============================] - 1s 911us/step - loss: 0.5277 - accuracy: 0.7410 - val_loss: 0.6313 - val_accuracy: 0.7291
Epoch 499/500
1437/1437 [==============================] - 1s 927us/step - loss: 0.5278 - accuracy: 0.7411 - val_loss: 0.6276 - val_accuracy: 0.7287
Epoch 500/500
1437/1437 [==============================] - 1s 900us/step - loss: 0.5278 - accuracy: 0.7415 - val_loss: 0.6322 - val_accuracy: 0.7301


In [141]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

354/354 - 0s - loss: 0.6322 - accuracy: 0.7301 - 204ms/epoch - 577us/step
Loss: 0.6321508288383484, Accuracy: 0.7300998568534851
